## Prepare TGW input data for prediction

1. Import TGW data
2. Process TGW data: rename columns to match training data, add timezone, add useful numerical time features (month, day, hour etc.), add weather features (e.g., last 24h temp average)
3. Save as dictionary

### Import Packages

In [1]:
import numpy as np
import pyarrow.parquet as pq
import joblib
import scipy
import pandas as pd
import time
import os
import re
import glob
from datetime import datetime, timedelta
import yaml
import pprint
import pytz
from timezonefinder import TimezoneFinder
import matplotlib.pyplot as plt

from src import figure_ops
from src import input_ops
from src import model_ops
from src import aux_ops

### Load config file 

In [2]:
config_file_name = 'config1'; config_path = f"config/{config_file_name}.yaml"; config = input_ops.load_config(config_path)
# pprint.pprint(config, sort_dicts=False)
print(f"TGW_scenario:{config['TGW_scenario']} \nTGW_years:{config['TGW_weather_years']} \ninput_data_prediction_path: {config['input_data_prediction_path']}")

TGW_scenario:rcp85hotter 
TGW_years:['2058'] 
input_data_prediction_path: /nfs/turbo/seas-mtcraig-climate/Aviad/load_prediction/results/data/prediction/input/TGW_weather


### Print output tree directory to see previous saved joblib files

In [3]:
joblib_tree_path =  config['input_data_prediction_path']
print(f"printing joblib tree of path: {joblib_tree_path}")
aux_ops.print_joblib_tree(joblib_tree_path)

printing joblib tree of path: /nfs/turbo/seas-mtcraig-climate/Aviad/load_prediction/results/data/prediction/input/TGW_weather
TGW_weather/
  Austin/
    historical/
      └── TGW_weather_2017.joblib
      └── TGW_weather_2018.joblib
    rcp45cooler/
      └── TGW_weather_2058.joblib
    rcp45hotter/
      └── TGW_weather_2058.joblib
    rcp85cooler/
      └── TGW_weather_2058.joblib
  Concord/
    historical/
      └── TGW_weather_2017.joblib
      └── TGW_weather_2018.joblib
    rcp45cooler/
      └── TGW_weather_2058.joblib
    rcp45hotter/
      └── TGW_weather_2058.joblib
    rcp85cooler/
      └── TGW_weather_2058.joblib
  Greensboro/
    historical/
      └── TGW_weather_2017.joblib
      └── TGW_weather_2018.joblib
    rcp45cooler/
      └── TGW_weather_2058.joblib
    rcp45hotter/
      └── TGW_weather_2058.joblib
    rcp85cooler/
      └── TGW_weather_2058.joblib
  SanFrancisco/
    historical/
      └── TGW_weather_2017.joblib
      └── TGW_weather_2018.joblib
    rcp45cooler

### Import, process and save TGW data

In [4]:
# Import, process and save load and weather data (run only once per dataset, e.g., run again if want to add new regions or years)
start_time = time.time()

input_data_prediction_path = config['input_data_prediction_path']
TGW_weather_years = config['TGW_weather_years']

# List of TGW locations to create data frames for
TGW_locations = ['Concord','SanFrancisco','Austin','Greensboro']   

# Loop through all TGW weather years and locations
for TGW_weather_year in TGW_weather_years:
    # Set TGW scenario, historical for years 1980-2019, chosen RSP for years 2020-2099
    if int(TGW_weather_year) < 2020:
        TGW_scenario = 'historical'
    else:
        TGW_scenario = config['TGW_scenario']
    for TGW_location in TGW_locations:
        # Import raw TGW weather data
        weather_data_path = f"/nfs/turbo/seas-mtcraig-climate/TGW/TGW_Distribution_for_Aviad/Yearly/{TGW_location}/{TGW_scenario}/tgw_wrf_{TGW_scenario}_hourly_{TGW_weather_year}.csv"
        weather_df = input_ops.import_TGW_weather_data(weather_data_path, TGW_weather_year, TGW_location, start_month = 1, end_month = 12)
        # Add input features
        weather_df = model_ops.add_features_X_columns_D(weather_df)
        # Save data frame
        TGW_weather_df_save_path = f"{config['input_data_prediction_path']}/{TGW_location}/{TGW_scenario}/"
        # Check if directory exists - if not,create it. 
        if not os.path.exists(TGW_weather_df_save_path):
            os.makedirs(TGW_weather_df_save_path, exist_ok=True)
        # Save
        print(f'saving joblib for {TGW_location} {TGW_weather_year}  {TGW_scenario}')
        joblib.dump(weather_df, os.path.join(TGW_weather_df_save_path, f"TGW_weather_{TGW_weather_year}.joblib"))   

end_time = time.time(); print(f"Runtime for loading data: {(end_time - start_time) / 60:.2f} minutes")

saving joblib for Concord 2058  rcp85hotter
saving joblib for SanFrancisco 2058  rcp85hotter
saving joblib for Austin 2058  rcp85hotter
saving joblib for Greensboro 2058  rcp85hotter
Runtime for loading data: 0.05 minutes


In [10]:
TGW_weather_df_save_path

'/nfs/turbo/seas-mtcraig-climate/Aviad/load_prediction/results/data/prediction/input/TGW_weather/Greensboro/historical/'

### Load weather data frame

In [5]:
TGW_location = 'Greensboro'
TGW_scenario = config['TGW_scenario']
TGW_weather_year = config['TGW_weather_years'][0]
if int(TGW_weather_year) < 2020:
    TGW_scenario = 'historical'
else:
    TGW_scenario = config['TGW_scenario']
TGW_weather_df_save_path = f"{config['input_data_prediction_path']}/{TGW_location}/{TGW_scenario}/"
loaded_weather_df = joblib.load(os.path.join(TGW_weather_df_save_path, f"TGW_weather_{TGW_weather_year}.joblib"))

In [6]:
loaded_weather_df.columns

Index(['date_time', 'year', 'month', 'day', 'hour', 'weekday', 'weekend',
       'Relative Humidity [%]', 'Dry Bulb Temperature [°C]',
       'Global Horizontal Radiation [W/m2]', 'Wind Speed [m/s]',
       'Wind Direction [Deg]', 'last 24h avg Temp', 'last 12h avg Temp',
       'Minus 1h Temp', 'Minus 3h Temp', 'Minus 6h Temp', 'Minus 12h Temp',
       'Minus 24h Temp', 'sin hour', 'cos hour', 'temp times sin hour',
       'temp times cos hour'],
      dtype='object')

___

### DEBUG Day Light Savings: March 11th missess 2am and Nov 4th has 1am twice (Day light saving). Temporary Solution: import date time column from resstock.

In [45]:
input_data_prediction_path = config['input_data_prediction_path']
TGW_weather_years = config['TGW_weather_years']

# List of TGW locations to create data frames for
TGW_locations = ['Concord','SanFrancisco','Austin','Greensboro']  
TGW_location = 'Greensboro'
TGW_weather_year = '2018'
TGW_scenario = 'historical'
weather_data_path = f"/nfs/turbo/seas-mtcraig-climate/TGW/TGW_Distribution_for_Aviad/Yearly/{TGW_location}/{TGW_scenario}/tgw_wrf_{TGW_scenario}_hourly_{TGW_weather_year}.csv"
# weather_df = input_ops.import_TGW_weather_data(weather_data_path, TGW_weather_year, TGW_location, start_month = 1, end_month = 12)

weather_df = pd.read_csv(weather_data_path)

weather_df

### Rename columns to match training dataframe (with weather data from resstock) ###
weather_df.rename(columns={weather_df.columns[0]: "Index"}, inplace=True)
weather_df.rename(columns={weather_df.columns[1]: "Relative Humidity [%]"}, inplace=True)
weather_df.rename(columns={weather_df.columns[2]: "Dry Bulb Temperature [°C]"}, inplace=True)
weather_df.rename(columns={weather_df.columns[3]: "Global Horizontal Radiation [W/m2]"}, inplace=True)
weather_df.rename(columns={weather_df.columns[4]: "Wind Speed [m/s]"}, inplace=True)
weather_df.rename(columns={weather_df.columns[5]: "Wind Direction [Deg]"}, inplace=True)
weather_df["Dry Bulb Temperature [°C]"] = weather_df["Dry Bulb Temperature [°C]"] - 273.15 # Convert from Kelvin to Celcuis



weather_df = weather_df.drop('Index', axis=1) 

### Get the time zone for TGW locations coordinates ###
#  latitude and longitude for TGW cities for time zone correction 
location_coordinates = {
    "Greensboro": (36.0608, -80.0003),
    "Austin": (30.4196, -97.8095),
    "SanFrancisco": (37.7083, -122.4074),
    "Concord": (37.7083, -122.4074),
}
# Define the location (either a city name or coordinates)
latitude, longitude = location_coordinates[TGW_location]
# Get the time zone for the given coordinates
tf = TimezoneFinder(); timezone_str = tf.timezone_at(lng=longitude, lat=latitude)
datetime_index = pd.date_range(start=f"{TGW_weather_year}-01-01 00:00:00",periods=len(weather_df),freq='H',tz=timezone_str) 
# Replace the first column with the generated datetime values
weather_df["date_time"] = datetime_index

# Generate datetime values for the entire year with hourly resolution


### Load Resstock data ###
# Load dictionary load & weather data of year-city-region-building_type combinations
loaded_input_data_dict = joblib.load(os.path.join(config["input_data_training_path"], f"input_data_dict.joblib"))
resstock_weather_year =  '2018'
city1 = 'AUS' # GSO: ["rural", "industrial", "urban-suburban"]    AUS: ["P1R", "P1U", "P2U"]    SFO: ["P1R", "P1U", "P2U"]
region = 'P1R'
# region = 'rural'
df_resstock1 = loaded_input_data_dict[(resstock_weather_year, city1, region, 'regional', 'res')]
df_resstock1['date_time'] = df_resstock1.index; 

# display(df_resstock1.head())

weather_df = weather_df.reset_index(drop=True)
df_resstock1 = df_resstock1.reset_index(drop=True)

# display(weather_df.head())
# Extract UTC offset in hours 
utc_offset_hours = int(weather_df["date_time"].iloc[0].utcoffset().total_seconds() / 3600)

# cyclically shift all columns except date_time
# Make a copy of the DataFrame to avoid modifying the original
weather_df_shifted = weather_df.copy()
# Get list of all columns except 'date_time'
columns_to_shift = [col for col in weather_df.columns if col != "date_time"]


# Apply cyclic shift to each of those columns
weather_df_shifted[columns_to_shift] = np.roll(weather_df_shifted[columns_to_shift], utc_offset_hours,axis=0)

weather_df = weather_df_shifted

weather_df_shifted["resstock_date_time"] = df_resstock1["date_time"]


### Add useful time numerical features ###
# Convert datetime column to useful numerical features
weather_df["year"] = weather_df["date_time"].dt.year
weather_df["month"] = weather_df["date_time"].dt.month
weather_df["day"] = weather_df["date_time"].dt.day
weather_df["hour"] = weather_df["date_time"].dt.hour
weather_df["weekday"] = weather_df["date_time"].dt.weekday  # Monday = 0, Sunday = 6
weather_df["weekend"] = (weather_df["date_time"].dt.weekday >= 5).astype(int)  # 1 if weekend, else 0
weather_df = weather_df.reset_index(drop=True) # Reset indices

weather_df.rename(columns={
    'date_time': 'date_time_DST',
}, inplace=True)

# Put time columns first
cols_to_move = ['resstock_date_time', 'date_time_DST', 'year', 'month', 'day', 'hour', 'weekday', 'weekend'] # Columns to move to the front
remaining_cols = [col for col in weather_df.columns if col not in cols_to_move] # Get the remaining columns (i.e., all except those in cols_to_move)
new_order = cols_to_move + remaining_cols # Define new column order
weather_df = weather_df[new_order] # Reorder DataFrame


weather_df.head()
# weather_df = weather_df[(weather_df['month'] >= start_month) & (weather_df['month'] <= end_month)] # Filter data to only include rows within the specified month range

# display(weather_df[(weather_df['day']==11) & (weather_df['month']==3)].head(4))
# display(weather_df[(weather_df['day']==4) & (weather_df['month']==11)].head(4))

,resstock_date_time,date_time_DST,year,month,day,hour,weekday,weekend,Relative Humidity [%],Dry Bulb Temperature [°C],Global Horizontal Radiation [W/m2],Wind Speed [m/s],Wind Direction [Deg]
0,2018-01-01 00:00:00,2018-01-01 00:00:00-05:00,2018,1,1,0,0,0,55.522125,-8.19984,0.0,3.379309,26.652569
1,2018-01-01 01:00:00,2018-01-01 01:00:00-05:00,2018,1,1,1,0,0,58.502052,-8.58893,0.0,2.639000,29.261845
2,2018-01-01 02:00:00,2018-01-01 02:00:00-05:00,2018,1,1,2,0,0,60.381073,-8.98167,0.0,2.513349,33.948437
3,2018-01-01 03:00:00,2018-01-01 03:00:00-05:00,2018,1,1,3,0,0,61.522080,-9.43137,0.0,3.218416,18.827988
4,2018-01-01 04:00:00,2018-01-01 04:00:00-05:00,2018,1,1,4,0,0,67.496970,-10.08630,0.0,2.541826,353.366300
